
# 🌼 자연어처리 기초 - 4차시(24.11.13)

---


#### 1. RNN을 이용하여 텍스트 생성하기

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [ ]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [ ]:
tokenizer.word_index

# 가장 많이 나오는 단어가 위에 저장되고, 빈도가 같다면 순서대로

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [ ]:
vocab_size = len(tokenizer.word_index) +1
vocab_size

12

In [ ]:
sequences = []
for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    # print(encoded)
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        # i=1일 때, sequence=[2, 3]
        # i=2일 때, sequence=[2, 3, 1]
        sequences.append(sequence)

print(f'학습에 사용할 샘플의 개수: {len(sequences)}')

학습에 사용할 샘플의 개수: 11


In [ ]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [ ]:
lengths = []
for i in sequences:
    lengths.append(len(i))

In [ ]:
max_len = max(lengths)
print(f'샘플 중 최대 길이: {max_len}')

샘플 중 최대 길이: 6


In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [ ]:
sequences = np.array(sequences)
sequences  # 타입 에러의 위험을 피하기 위해 numpy의 array로 변환

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [ ]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [ ]:
y = to_categorical(y, num_classes=vocab_size)  # 원핫인코딩 됨
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

#### 2) 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10
hidden_units = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # vocab size -> embedding dim으로 차원을 바꿔주겠다
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
# vocab_size = 12인데 단어 11개 + 1 했던 word_index 값
# +1은 자리 차지를 위해 앞에 0을 채워준 것으로 실제 예측을 수행할 때에는 0은 단어가 아니므로 무시되어 출력 값으로 0이 나오지는 않는다
# 11개의 단어일 때 / 12개를 input으로 넣는 이유: 내가 예측을 위해 넣을 단어를 포함하기 위해

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=200, verbose=2)  # verbose=2 수치화 시각화

Epoch 1/200
1/1 - 1s - loss: 2.5066 - accuracy: 0.0000e+00 - 924ms/epoch - 924ms/step
Epoch 2/200
1/1 - 0s - loss: 2.4960 - accuracy: 0.0909 - 2ms/epoch - 2ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4858 - accuracy: 0.0909 - 3ms/epoch - 3ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4760 - accuracy: 0.0909 - 4ms/epoch - 4ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4664 - accuracy: 0.0909 - 3ms/epoch - 3ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4569 - accuracy: 0.0909 - 3ms/epoch - 3ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4475 - accuracy: 0.0909 - 3ms/epoch - 3ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4381 - accuracy: 0.0909 - 3ms/epoch - 3ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4286 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 10/200
1/1 - 0s - loss: 2.4188 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 11/200
1/1 - 0s - loss: 2.4088 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3985 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3877 - accuracy: 0.45

In [ ]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for i in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)  # result 사이에서 가장 높은 확률을 가진 단어만 다시 result에 넣어준다

        for word, index in tokenizer.word_index.items():
        # (말이, 1)
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    
    return sentence

In [ ]:
sentence_generation(model, tokenizer, '경마장에', 1)

'경마장에 있는'

In [ ]:
sentence_generation(model, tokenizer, '경마장에', 4)

'경마장에 있는 말이 뛰고 있다'

In [ ]:
sentence_generation(model, tokenizer, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

In [ ]:
sentence_generation(model, tokenizer, '고와야', 2)

'고와야 말이 말이'

### 2. LSTM을 이용하여 텍스트 생성하기

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from string import punctuation
import pandas as pd

In [ ]:
df = pd.read_csv('./data/ArticlesApril2018.csv')
df.head(30)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
5,5adf341f068401528a2aa5e8,1233,By NICHOLAS FANDOS and MICHAEL D. SHEAR,article,Unknown,"['United States Politics and Government', 'Vet...",68,Washington,0,2018-04-24 13:41:49,Politics,The Senate Veterans Affairs Committee is exami...,The New York Times,News,https://www.nytimes.com/2018/04/24/us/politics...
6,5adf2a0f068401528a2aa5c8,871,By ADAM NAGOURNEY and CHARLES McDERMID,article,Unknown,[],0,NYTNow,0,2018-04-24 12:58:53,Unknown,Tuesday: Rent control appears headed to the ba...,The New York Times,briefing,https://www.nytimes.com/2018/04/24/us/californ...
7,5adf26cf068401528a2aa5c1,938,By NIRAJ CHOKSHI,article,Unknown,"['Presidential Election of 2016', 'Proceedings...",68,Express,0,2018-04-24 12:45:00,Politics,"A new study suggests that the white, Christian...",The New York Times,News,https://www.nytimes.com/2018/04/24/us/politics...
8,5adf2108068401528a2aa5b3,730,By JOHN SCHWARTZ,article,How a Bag of Texas Dirt Became a Times Tradition,"['Texas', 'Soil', 'Pregnancy and Childbirth', ...",68,Insider,2,2018-04-24 12:20:21,Unknown,What can you do when you want your child to be...,The New York Times,News,https://www.nytimes.com/2018/04/24/insider/how...
9,5adedaa8068401528a2aa4e6,482,By NATALIE PROULX,article,Is School a Place for Self-Expression?,[],66,Learning,0,2018-04-24 11:21:04,Unknown,Are you encouraged to question and articulate ...,The New York Times,News,https://www.nytimes.com/2018/04/24/learning/is...


In [ ]:
print(f'열의 개수: {len(df.columns)}')
print(df.columns)

열의 개수: 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [ ]:
df['headline'].isnull().values.any()

False

In [ ]:
headline = []
headline.extend(df.headline.values)
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [ ]:
print(f'총 샘플의 개수 (노이즈 제거 전): {len(headline)}')

총 샘플의 개수 (노이즈 제거 전): 1324


In [ ]:
clean_headline = []
for h in headline:
    if h != 'Unknown':
        clean_headline.append(h)

print(f'총 샘플의 개수 (노이즈 제거 후): {len(clean_headline)}')

총 샘플의 개수 (노이즈 제거 후): 1214


In [ ]:
clean_headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

#### 전처리

In [ ]:
def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode('utf-8').decode('ascii', 'ignore')
    # ascii: 단순하고 영어 텍스트에 효과적
    # utf-8: 다국어와 기호 처리 가능
    # 해당 데이터 셋은 영어로 되어있기 때문에 우선적으로 다국어와 기호를 처리한 후
    # 단순하고 영어 텍스트에 효과적인 아스키 문자로 변환
    # ignore: ascii에 없는 문자들은 무시 (특수기호 등)
    result = ''
    for word in preprocessed_sentence:
        if word not in punctuation: # 특수기호가 아닌 것들
            result += word
    
    return result.lower()

In [ ]:
preprocessed_headline = []
# 전처리된 헤드라인들을 담아주기

for word in clean_headline:
    preprocessed_sentence = repreprocessing(word)
    preprocessed_headline.append(preprocessed_sentence)

preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)

In [ ]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'is': 8,
 'on': 9,
 'with': 10,
 'trump': 11,
 'as': 12,
 'at': 13,
 'new': 14,
 'how': 15,
 'from': 16,
 'it': 17,
 'an': 18,
 'that': 19,
 'be': 20,
 'season': 21,
 'us': 22,
 'you': 23,
 'its': 24,
 'what': 25,
 'episode': 26,
 'can': 27,
 'your': 28,
 'not': 29,
 'he': 30,
 'now': 31,
 'his': 32,
 'are': 33,
 'teaching': 34,
 'war': 35,
 'out': 36,
 'no': 37,
 'was': 38,
 'by': 39,
 'trumps': 40,
 'has': 41,
 'over': 42,
 'may': 43,
 'into': 44,
 'why': 45,
 'more': 46,
 'we': 47,
 'who': 48,
 'about': 49,
 'recap': 50,
 'activities': 51,
 '1': 52,
 'just': 53,
 'do': 54,
 'women': 55,
 'when': 56,
 'syria': 57,
 'trade': 58,
 'i': 59,
 '2': 60,
 'or': 61,
 'will': 62,
 'this': 63,
 'have': 64,
 'president': 65,
 'but': 66,
 'home': 67,
 'up': 68,
 'long': 69,
 'one': 70,
 'off': 71,
 'facebook': 72,
 'house': 73,
 'gop': 74,
 'our': 75,
 'case': 76,
 'they': 77,
 'life': 78,
 'end': 79,
 'right': 80,
 'some':

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(f'단어 집합의 크기: {vocab_size}')

단어 집합의 크기: 3494


In [ ]:
sequences = []
for sentence in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        # 인코딩된 문장이 [3, 5, 6, 7] 이라면
        # 첫 번쨰 반복에서는 [3, 5]
        # 두 번째 반복에서는 [3, 5, 6]
        sequences.append(sequence)

In [ ]:
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [ ]:
lengths = []
for i in sequences:
    lengths.append(len(i))

max_len = max(lengths)
print(f'샘플의 최대 길이: {max_len}')

샘플의 최대 길이: 24


In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [ ]:
sequences = np.array(sequences)

In [ ]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [ ]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [ ]:
print(y[:3])

[ 269  371 1115]


In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
y[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#### 2) 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X.shape

(7803, 23)

In [136]:
embedding_dim = 10
# embedding_dim이 커질수록 더 세밀한 표현 가능
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
hist = model.fit(X, y, epochs=300, verbose=2, callbacks=[es])

Epoch 1/300
244/244 - 7s - loss: 7.6485 - accuracy: 0.0291 - 7s/epoch - 28ms/step
Epoch 2/300
244/244 - 6s - loss: 7.1177 - accuracy: 0.0302 - 6s/epoch - 23ms/step
Epoch 3/300
244/244 - 5s - loss: 6.9824 - accuracy: 0.0342 - 5s/epoch - 21ms/step
Epoch 4/300
244/244 - 5s - loss: 6.8649 - accuracy: 0.0384 - 5s/epoch - 20ms/step
Epoch 5/300
244/244 - 5s - loss: 6.7242 - accuracy: 0.0437 - 5s/epoch - 21ms/step
Epoch 6/300
244/244 - 6s - loss: 6.5592 - accuracy: 0.0442 - 6s/epoch - 23ms/step
Epoch 7/300


KeyboardInterrupt: 

In [137]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for i in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')  # 내가 입력한 한 단어를 제외한 max_len-1

        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        # 단어 집합 사전의 인덱스 값으로 result가 저장된다

        # tokenizer.word_index {단어: 인덱스, 단어: 인덱스}
        for word, index in tokenizer.word_index.items():
            if result == index:
                break
        
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    
    sentence = init_word + sentence

    return sentence

In [ ]:
sentence_generation(model, tokenizer, 'i', 15)

'i cant jump ship from facebook yet the past of the iceberg of aims more truth'

In [ ]:
loss, acc = model.evaluate(X, y)
print(f'loss: {loss}, 정확도: {acc}')

244/244 [==============================] - 2s 10ms/step - loss: 0.2772 - accuracy: 0.9245
loss: 0.2772088944911957, 정확도: 0.9245162010192871


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
epochs = range(1, len(hist.history['accuracy'])+1)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, hist.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.subplot(1, 2, 2)
plt.plot(epochs, hist.history['accuracy'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.show()

### 딥러닝 분류

In [138]:
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path:", path)

c:\Users\RMARKET\anaconda3\envs\tensor_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path: C:\Users\RMARKET\.cache\kagglehub\datasets\uciml\sms-spam-collection-dataset\versions\1


In [139]:
data = pd.read_csv('C:/Users/RMARKET/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1/spam.csv', encoding='latin1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [140]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

In [141]:
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [142]:
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
data[:5]

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [143]:
data.isnull().values.any()

False

In [144]:
data['v2'].nunique()

5169

In [145]:
data.drop_duplicates(subset=['v2'], inplace=True)
print(f'총 샘플의 개수: {len(data)}')

총 샘플의 개수: 5169


In [146]:
data.groupby('v1').size().reset_index(name='count')

,v1,count
0,0,4516
1,1,653


In [147]:
X_data = data['v2']
y_data = data['v1']

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)  # stratify: 0과 1이 균일하게 들어간다

In [149]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_encoded = tokenizer.texts_to_sequences(X_train)
print(X_train_encoded[:5])

[[102, 1, 210, 230, 3, 17, 39], [1, 59, 8, 427, 17, 5, 137, 2, 2326], [157, 180, 12, 13, 98, 93, 47, 9, 40, 3485, 247, 8, 7, 87, 6, 80, 1312, 5, 3486, 7, 2327, 11, 660, 306, 20, 25, 467, 708, 1028, 203, 129, 193, 800, 2328, 23, 1, 144, 71, 2, 111, 78, 43, 2, 130, 11, 800, 186, 122, 1512], [1, 1154, 13, 104, 292], [222, 622, 857, 540, 623, 22, 23, 83, 10, 47, 6, 257, 32, 6, 26, 64, 936, 407]]


In [150]:
word_to_index = tokenizer.word_index
print(word_to_index)

{'i': 1, 'to': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'in': 8, 'is': 9, 'me': 10, 'my': 11, 'for': 12, 'your': 13, 'it': 14, 'of': 15, 'have': 16, 'on': 17, 'call': 18, 'that': 19, 'are': 20, '2': 21, 'now': 22, 'so': 23, 'but': 24, 'not': 25, 'can': 26, 'or': 27, "i'm": 28, 'get': 29, 'at': 30, 'do': 31, 'if': 32, 'be': 33, 'will': 34, 'just': 35, 'with': 36, 'we': 37, 'no': 38, 'this': 39, 'ur': 40, 'up': 41, '4': 42, 'how': 43, 'gt': 44, 'lt': 45, 'go': 46, 'when': 47, 'from': 48, 'what': 49, 'ok': 50, 'out': 51, 'know': 52, 'free': 53, 'all': 54, 'like': 55, 'then': 56, 'got': 57, 'good': 58, 'am': 59, 'time': 60, 'was': 61, 'come': 62, 'its': 63, 'love': 64, 'want': 65, 'text': 66, 'he': 67, 'only': 68, 'there': 69, 'day': 70, 'need': 71, 'going': 72, 'lor': 73, 'send': 74, 'one': 75, 'as': 76, 'home': 77, 'about': 78, 'back': 79, 'still': 80, 'k': 81, 'see': 82, 'txt': 83, 'by': 84, 'da': 85, 'stop': 86, 'r': 87, 'any': 88, 'tell': 89, 'dont': 90, "i'll": 91, "don't": 9

In [151]:
vocab_size = len(word_to_index) + 1
print(f'단어 집합의 크기: {vocab_size}')

단어 집합의 크기: 7822


In [152]:
max_lengths = 0
for sample in X_train_encoded:
    lengths =  len(sample)
    if lengths > max_lengths:
        max_lengths = lengths
print(f'메일 최대 길이: {max_lengths}')

메일 최대 길이: 189


In [153]:
lengths = []
for sample in X_data:
    lengths.append(len(sample))

print(lengths)

[111, 29, 155, 49, 61, 148, 77, 160, 158, 154, 109, 136, 156, 196, 35, 149, 26, 81, 58, 156, 41, 49, 53, 88, 57, 144, 30, 134, 75, 64, 130, 189, 29, 84, 159, 123, 47, 28, 27, 155, 82, 142, 172, 19, 72, 32, 45, 31, 67, 148, 58, 124, 80, 291, 120, 76, 161, 34, 22, 40, 108, 48, 25, 56, 110, 153, 124, 161, 80, 34, 46, 29, 45, 42, 20, 43, 73, 50, 42, 76, 22, 32, 32, 36, 14, 55, 121, 144, 42, 41, 62, 195, 141, 139, 107, 125, 33, 51, 184, 57, 81, 76, 183, 44, 95, 43, 82, 115, 30, 40, 31, 96, 159, 143, 156, 153, 72, 86, 144, 157, 53, 156, 52, 40, 20, 244, 22, 107, 28, 9, 39, 25, 126, 162, 38, 34, 46, 155, 86, 33, 27, 158, 42, 25, 48, 159, 84, 33, 30, 45, 59, 25, 384, 28, 27, 157, 124, 146, 115, 64, 85, 152, 155, 51, 157, 74, 67, 59, 50, 94, 33, 105, 61, 65, 26, 146, 66, 126, 159, 23, 65, 24, 26, 152, 34, 149, 55, 88, 72, 185, 37, 111, 92, 28, 28, 64, 131, 40, 28, 84, 174, 24, 25, 64, 28, 86, 39, 73, 26, 23, 24, 31, 58, 48, 41, 32, 159, 25, 161, 119, 143, 69, 137, 30, 165, 34, 109, 37, 33, 48, 

In [ ]:
plt.hist(lengths, bins=50)
plt.xlabel('lengths')
plt.ylabel('count')
plt.show()

In [154]:
X_train_padding = pad_sequences(X_train_encoded, maxlen=max_lengths)

In [155]:
X_train_padding

array([[   0,    0,    0, ...,    3,   17,   39],
       [   0,    0,    0, ...,  137,    2, 2326],
       [   0,    0,    0, ...,  186,  122, 1512],
       ...,
       [   0,    0,    0, ...,    2,   18, 1604],
       [   0,    0,    0, ..., 1297,  560,  122],
       [   0,    0,    0, ...,   12, 7820, 7821]])

In [156]:
embedding_dim = 8
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # 차원 축소, 단어의 의미 관계
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])  # rmsprop: 보폭 크게 지나갔다가 중요해보이면 보폭 축소

history = model.fit(X_train_padding, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
52/52 [==============================] - 3s 28ms/step - loss: 0.4501 - accuracy: 0.8271 - val_loss: 0.3547 - val_accuracy: 0.8779
Epoch 2/5
52/52 [==============================] - 1s 24ms/step - loss: 0.2340 - accuracy: 0.9281 - val_loss: 0.1283 - val_accuracy: 0.9686
Epoch 3/5
52/52 [==============================] - 1s 24ms/step - loss: 0.1026 - accuracy: 0.9710 - val_loss: 0.1007 - val_accuracy: 0.9686
Epoch 4/5
52/52 [==============================] - 1s 23ms/step - loss: 0.0631 - accuracy: 0.9840 - val_loss: 0.0753 - val_accuracy: 0.9758
Epoch 5/5
52/52 [==============================] - 1s 24ms/step - loss: 0.0454 - accuracy: 0.9861 - val_loss: 0.0661 - val_accuracy: 0.9794


In [157]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padding = pad_sequences(X_test_encoded, maxlen=max_lengths)

accuracy = model.evaluate(X_test_padding, y_test)[1]
print(accuracy)

33/33 [==============================] - 0s 7ms/step - loss: 0.0701 - accuracy: 0.9816
0.9816247820854187


In [158]:
p = model.predict(X_test_padding)
p

33/33 [==============================] - 0s 6ms/step


array([[0.00706928],
       [0.00832272],
       [0.99217236],
       ...,
       [0.00427008],
       [0.9949901 ],
       [0.00824471]], dtype=float32)

In [ ]:
epochs = range(1, len(history.history['accuracy'])+1)

plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential


## 🌼마무리 문제

스팸메일 분류의 정확도를 더 개선해보세요!  

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

embedding_dim = 8
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # 차원 축소, 단어의 의미 관계
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # rmsprop: 보폭 크게 지나갔다가 중요해보이면 보폭 축소

es = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(X_train_padding, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/100
52/52 [==============================] - 4s 42ms/step - loss: 0.4102 - accuracy: 0.8752 - val_loss: 0.3578 - val_accuracy: 0.8851
Epoch 2/100
52/52 [==============================] - 1s 27ms/step - loss: 0.2956 - accuracy: 0.8945 - val_loss: 0.1804 - val_accuracy: 0.9637
Epoch 3/100
52/52 [==============================] - 1s 29ms/step - loss: 0.1126 - accuracy: 0.9746 - val_loss: 0.0976 - val_accuracy: 0.9746
Epoch 4/100
52/52 [==============================] - 1s 28ms/step - loss: 0.0583 - accuracy: 0.9888 - val_loss: 0.0821 - val_accuracy: 0.9770
Epoch 5/100
52/52 [==============================] - 1s 27ms/step - loss: 0.0404 - accuracy: 0.9918 - val_loss: 0.0790 - val_accuracy: 0.9770
Epoch 6/100
52/52 [==============================] - 1s 27ms/step - loss: 0.0293 - accuracy: 0.9930 - val_loss: 0.0874 - val_accuracy: 0.9722
Epoch 7/100
52/52 [==============================] - 1s 28ms/step - loss: 0.0265 - accuracy: 0.9937 - val_loss: 0.0678 - val_accuracy: 0.9807
Epoch 

In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padding = pad_sequences(X_test_encoded, maxlen=max_lengths)

accuracy = model.evaluate(X_test_padding, y_test)[1]
print(accuracy)

33/33 [==============================] - 0s 7ms/step - loss: 0.0846 - accuracy: 0.9787
0.978723406791687


In [ ]:
epochs = range(1, len(history.history['accuracy'])+1)

plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()